In [1]:
import pandas as pd

In [2]:
DATA = './data/nyc-taxi/'

In [4]:
%%time

df = pd.read_parquet(
    DATA,
    columns=[
        'VendorID',
        'tpep_pickup_datetime',
        'tpep_dropoff_datetime',
        'passenger_count',
        'trip_distance',
        'RatecodeID',
        'store_and_fwd_flag',
        'PULocationID',
        'DOLocationID',
        'payment_type',
        'fare_amount',
        'extra',
        'mta_tax',
        'tip_amount',
        'tolls_amount',
        'improvement_surcharge',
        'total_amount',
        'congestion_surcharge',
    ]  # the inconsistent treatment of airport_fee type causes error so drop it
)

CPU times: user 17 s, sys: 12.1 s, total: 29.1 s
Wall time: 23.2 s


In [5]:
df.shape

(91809949, 18)

In [6]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1,2020-01-01 00:28:15,2020-01-01 00:33:03,1.0,1.2,1.0,N,238,239,1,6.0,3.0,0.5,1.47,0.0,0.3,11.27,2.5
1,1,2020-01-01 00:35:39,2020-01-01 00:43:04,1.0,1.2,1.0,N,239,238,1,7.0,3.0,0.5,1.50,0.0,0.3,12.30,2.5
2,1,2020-01-01 00:47:41,2020-01-01 00:53:52,1.0,0.6,1.0,N,238,238,1,6.0,3.0,0.5,1.00,0.0,0.3,10.80,2.5
3,1,2020-01-01 00:55:23,2020-01-01 01:00:14,1.0,0.8,1.0,N,238,151,1,5.5,0.5,0.5,1.36,0.0,0.3,8.16,0.0
4,2,2020-01-01 00:01:58,2020-01-01 00:04:16,1.0,0.0,1.0,N,193,193,2,3.5,0.5,0.5,0.00,0.0,0.3,4.80,0.0


In [7]:
%%time

df['is_congested'] = df.congestion_surcharge.fillna(0) > 0

df.groupby('DOLocationID').agg(
    mean_fare=('fare_amount', 'mean'),
    count=('fare_amount', 'count'),
    congestion=('is_congested', 'mean')
).sort_values('count', ascending=False)

CPU times: user 1.4 s, sys: 2.44 s, total: 3.84 s
Wall time: 8.71 s


,mean_fare,count,congestion
DOLocationID,,,
236,9.600091,4109242,0.966388
237,9.118917,3845516,0.975696
161,10.634796,3165651,0.971493
170,10.632019,2730697,0.967348
141,9.888821,2567317,0.970547
...,...,...,...
2,42.227734,128,0.054688
105,18.675581,43,0.767442
110,80.333333,3,0.000000
